In [ ]:
# DRL Liquidity Sweep - Model Evaluation

This notebook visualizes the results of a trained model's performance on test data.


In [ ]:
import json
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [ ]:
## Load Evaluation Results

First, let's load the results from our evaluation run.


In [ ]:
# Update these paths to match your evaluation output
EVAL_DIR = Path("../logs/evaluation")

# Load metrics
with open(EVAL_DIR / "metrics.json") as f:
    metrics = json.load(f)
    
# Load equity curve
equity_df = pd.read_csv(EVAL_DIR / "equity.csv")
equity_df["time"] = pd.to_datetime(equity_df["time"])

# Load trades
trades_df = pd.read_csv(EVAL_DIR / "trades.csv")
trades_df["entry_time"] = pd.to_datetime(trades_df["entry_time"])
trades_df["exit_time"] = pd.to_datetime(trades_df["exit_time"])


In [ ]:
## Performance Metrics

Let's look at the key performance metrics.


In [ ]:
print("Performance Metrics:")
print(f"Total Return: {metrics['total_return']:.2%}")
print(f"Sharpe Ratio: {metrics['sharpe_ratio']:.2f}")
print(f"Sortino Ratio: {metrics['sortino_ratio']:.2f}")
print(f"MAR Ratio: {metrics['mar_ratio']:.2f}")
print(f"\nTrade Statistics:")
print(f"Total Trades: {metrics['total_trades']}")
print(f"Win Rate: {metrics['win_rate']:.2%}")
print(f"Average Win: {metrics['avg_win']:.4f}")
print(f"Average Loss: {metrics['avg_loss']:.4f}")
print(f"Profit Factor: {metrics['profit_factor']:.2f}")


In [ ]:
## Equity Curve and Drawdown

Let's plot the equity curve and underwater drawdown chart.


In [ ]:
# Calculate drawdown
rolling_max = equity_df["equity"].expanding().max()
drawdown = (equity_df["equity"] - rolling_max) / rolling_max

# Create subplot figure
fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                    subplot_titles=("Equity Curve", "Drawdown"),
                    vertical_spacing=0.1,
                    row_heights=[0.7, 0.3])

# Add equity curve
fig.add_trace(
    go.Scatter(x=equity_df["time"], y=equity_df["equity"],
               name="Equity", line=dict(color="blue")),
    row=1, col=1
)

# Add drawdown
fig.add_trace(
    go.Scatter(x=equity_df["time"], y=drawdown,
               name="Drawdown", fill="tonexty",
               line=dict(color="red")),
    row=2, col=1
)

fig.update_layout(
    height=800,
    showlegend=True,
    title_text="Portfolio Performance"
)

fig.show()


In [ ]:
## Attribution Analysis

Let's analyze performance by instrument.


In [ ]:
# Create attribution bar chart
attribution = pd.DataFrame(metrics["attribution"]).T

fig = go.Figure()

# Add total P&L bars
fig.add_trace(
    go.Bar(x=attribution.index, y=attribution["sum"],
           name="Total P&L")
)

# Add trade count bars
fig.add_trace(
    go.Bar(x=attribution.index, y=attribution["count"],
           name="Trade Count", yaxis="y2")
)

fig.update_layout(
    title="Performance Attribution by Instrument",
    yaxis=dict(title="Total P&L"),
    yaxis2=dict(title="Trade Count", overlaying="y", side="right"),
    barmode="group"
)

fig.show()


In [ ]:
## Trade Distribution

Finally, let's look at the distribution of trade P&Ls.


In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Histogram(x=trades_df["pnl"], nbinsx=50,
                 name="Trade P&L Distribution")
)

fig.update_layout(
    title="Distribution of Trade P&Ls",
    xaxis_title="P&L",
    yaxis_title="Count",
    showlegend=True
)

fig.show()
